## Цель ноутбука

Построить и обучить модель для распознавания рукописных цифр на базе датасета [MNIST](http://yann.lecun.com/exdb/mnist/), используя нейронные сети.

На примере этой задачи мы рассмотрим процесс обучения нейронных сетей, необходимые его составляющие, а также сможем сравнить новый подход со знакомым методом классического машинного обучения.  

Будем использовать фреймворк [PyTorch](https://pytorch.org).

### 1. Устанавливаем и импортируем необходимые библиотеки

In [ ]:
!pip install numpy
!pip install matplotlib
!pip install torch torchvision
!pip install pillow

In [1]:
import os
import numpy as np
import torch
from torch import nn
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor, Compose, Resize, Normalize

### 2. Подготавливаем данные

PyTorch предлагает свою версию датасета MNIST. Он возвращает готовый экземпляр класса [`torch.utils.data.Dataset`](https://pytorch.org/tutorials/beginner/basics/data_tutorial.html). Каждый семпл датасета — это пара (изображение, лейбл), где изображение имеет размер 28 × 28 и является объектом класса [`PIL.Image`](https://pillow.readthedocs.io/en/stable/reference/Image.html), а лейбл — число от 0 до 9, соответствующее цифре на изображении.

Так как модели работают с тензорами, мы делаем из `PIL.Image` `torch.Tensor` методом `ToTensor()` из `torchvision.transforms`. Кроме этого, мы центрируем и нормируем данные, чтобы на вход модели приходили числа от −1 до 1.

Датасет мы оборачиваем в `Dataloader`, чтобы получить итерируемый объект и позаботиться о группировке семплов в батчи.

In [3]:
train_dataset = MNIST('data/', train=True, download=True)
val_dataset = MNIST('data/', train=False)

print('Train:', len(train_dataset))
print('Valid:', len(val_dataset))

for i in range(10):
    img, lbl = train_dataset[i]
    print(lbl, img.size)
    display(img)

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 9912422/9912422 [00:03<00:00, 2539578.49it/s]


Extracting data/MNIST\raw\train-images-idx3-ubyte.gz to data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|████████████████████████████████████████████████████████████████████████| 28881/28881 [00:00<00:00, 240039.98it/s]


Extracting data/MNIST\raw\train-labels-idx1-ubyte.gz to data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|███████████████████████████████████████████████████████████████████| 1648877/1648877 [00:00<00:00, 1947697.31it/s]


Extracting data/MNIST\raw\t10k-images-idx3-ubyte.gz to data/MNIST\raw

Failed to download (trying next):
HTTP Error 403: Forbidden



100%|██████████████████████████████████████████████████████████████████████████████████████| 4542/4542 [00:00<?, ?it/s]

Extracting data/MNIST\raw\t10k-labels-idx1-ubyte.gz to data/MNIST\raw

Train: 60000
Valid: 10000


In [7]:
train_dataset[0]

(<PIL.Image.Image image mode=L size=28x28>, 5)

In [ ]:
transform = Compose([
    ToTensor(),
    Normalize([0.5], [0.5])
])

train_dataset = MNIST('data/', train=True, download=True, transform=transform)
val_dataset = MNIST('data/', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=1000, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=1000)

### 3. Строим модель

Любой класс модели должен наследоваться от `torch.nn.Module` и иметь метод `forward()` для вызова модели. Первым делом мы попробуем написать логистическую регрессию для мультиклассовой классификации, она же Softmax-регрессия.

Чтобы правильно написать метод `forward()`, нужно сразу понять, с каким лоссом мы будем учить нашу модель. Удобный вариант — кросс-энтропия [`torch.nn.CrossEntropyLoss()`](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html).

$$H(p,q) = -\sum_x p(x)\log q(x)$$

$$L = - \frac{1}{N} \sum_n \left( 1 * \log \frac{\exp{x_{n,y_n}}}{\sum_c \exp{x_{n,c}}} \right)$$

In [ ]:
class LogReg(nn.Module):
    def __init__(self, in_features, n_classes):
        super(LogReg, self).__init__()
        self.fc = nn.Linear(in_features, n_classes)

    def forward(self, x):
        return self.fc(x)

### 4. Обучаем модель

In [ ]:
model = LogReg(in_features=28*28, n_classes=10)
loss_f = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

n_epoch = 50
val_fre = 10

model.train()
for epoch in range(n_epoch):
    loss_sum = 0
    for step, (data, target) in enumerate(train_loader):
        data = data.flatten(start_dim=1)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_f(output, target)
        loss.backward()
        optimizer.step()

        loss_sum += loss.item()

    print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

    if (epoch+1) % val_fre == 0:
        model.eval()
        loss_sum = 0
        correct = 0
        for step, (data, target) in enumerate(val_loader):
            data = data.flatten(start_dim=1)
            with torch.no_grad():
                output = model(data)
                loss = loss_f(output, target)
            loss_sum += loss.item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        acc = correct / len(val_loader.dataset)
        print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
        model.train()

### 5. Сохраняем (и загружаем) модель

Если обучение модели не завершено, то [аналогичным образом](https://pytorch.org/tutorials/beginner/saving_loading_models.html) можно сохранить и оптимизатор, и scheduler.

In [ ]:
os.makedirs('checkpoints/', exist_ok=True)
torch.save(model.state_dict(), 'checkpoints/logreg.pth')

model = LogReg(in_features=28*28, n_classes=10)
model.load_state_dict(torch.load('checkpoints/logreg.pth'))

### 6. Рубрика «Эксперименты»

Упакуем обучение и валидацию в функции и попробуем заменить линейный слой на двухслойный перцептрон.

In [ ]:
def train(model, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre):
    model.train()
    for epoch in range(n_epoch):
        loss_sum = 0
        for step, (data, target) in enumerate(train_loader):
            data = data.flatten(start_dim=1)
            optimizer.zero_grad()
            output = model(data)
            loss = loss_f(output, target)
            loss.backward()
            optimizer.step()

            loss_sum += loss.item()

        print(f'Epoch: {epoch} \tLoss: {loss_sum / (step + 1):.6f}')

        if epoch % val_fre == 0:
            validate(model, val_loader)

def validate(model, val_loader):
    model.eval()
    loss_sum = 0
    correct = 0
    for step, (data, target) in enumerate(val_loader):
        data = data.flatten(start_dim=1)
        with torch.no_grad():
            output = model(data)
            loss = loss_f(output, target)
        loss_sum += loss.item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
    acc = correct / len(val_loader.dataset)
    print(f'Val Loss: {loss_sum / (step + 1):.6f} \tAccuracy: {acc}')
    model.train()

In [ ]:
class MLP(nn.Module):
    def __init__(self, in_features, hid_features, n_classes):
        super(MLP, self).__init__()
        self.fc1 = nn.Linear(in_features, hid_features)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hid_features, n_classes)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [ ]:
model_mlp = MLP(in_features=28*28, hid_features=1024, n_classes=10)
print(model_mlp)
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model_mlp.parameters(), lr=1e-1)

n_epoch = 20
val_fre = 10

In [ ]:
train(model_mlp, optimizer, loss_f, train_loader, val_loader, n_epoch, val_fre)
validate(model_mlp, val_loader)